## TEAM MEMBERS
#### KARTHIK SAI TWARAKAVI
#### YOGANANDA THEEGURU
#### VENKATASAI NALLAPATI

### Install PuLP Package

In [1]:
!pip install pulp
!pip install rich
import pulp
import pandas as pd

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2


### Adding Data

In [2]:
feedstock_names = ["Alkylate", "CCG", "SRG", "Isopentane"]
gasoline_names = ["A", "B", "C"]
feedstock_avail = [140000, 130000, 140000, 110000] #In Gallons
gasoline_req = [120000, 130000, 120000] #In Gallons
price_gasoline = [3, 3.5, 4] #Dollar per Gallon
price_feedstock = [4.5, 2.5, 2.25, 2.35] #Dollar per Gallon
reid = [5, 8, 4, 20] # Reid vapor pressure
low_tel = [98, 87, 83, 101] #Used for GAS A in milli liters/Gallon
high_tel = [107, 93, 89, 108] #Used for GAS B & GAS C in milli liters/Gallon


### Formulate Linear programming Model

In [3]:
prob = pulp.LpProblem("Gasoline_Blending", pulp.LpMaximize) ##declaring to use Linear programming.Problem is named as Gasoline Blending

### Defining decision Variables

In [4]:
x = pulp.LpVariable.dicts("x", ((i, j) for i in feedstock_names for j in gasoline_names), lowBound=0, cat='Continuous')

In [5]:
print(x) # Printing decision Variables

{('Alkylate', 'A'): x_('Alkylate',_'A'), ('Alkylate', 'B'): x_('Alkylate',_'B'), ('Alkylate', 'C'): x_('Alkylate',_'C'), ('CCG', 'A'): x_('CCG',_'A'), ('CCG', 'B'): x_('CCG',_'B'), ('CCG', 'C'): x_('CCG',_'C'), ('SRG', 'A'): x_('SRG',_'A'), ('SRG', 'B'): x_('SRG',_'B'), ('SRG', 'C'): x_('SRG',_'C'), ('Isopentane', 'A'): x_('Isopentane',_'A'), ('Isopentane', 'B'): x_('Isopentane',_'B'), ('Isopentane', 'C'): x_('Isopentane',_'C')}


In [6]:
# Total used feedstock constraints
for i, feedstock_name in enumerate(feedstock_names):
    prob += pulp.lpSum(x[(feedstock_name, j)] for j in gasoline_names) <= feedstock_avail[i], f"Feedstock_{feedstock_name}_constraint"

# Total made gasoline constraints and gasoline A >= gasoline B constraint
prob += pulp.lpSum(x[(i, "A")] for i in feedstock_names) >= gasoline_req[0], "Gasoline_A_constraint"
prob += pulp.lpSum(x[(i, "B")] for i in feedstock_names) >= gasoline_req[1], "Gasoline_B_constraint"
prob += pulp.lpSum(x[(i, "C")] for i in feedstock_names) >= gasoline_req[2], "Gasoline_C_constraint"
prob += pulp.lpSum(x[(i, "A")] for i in feedstock_names) >= pulp.lpSum(x[(i, "B")] for i in feedstock_names), "Gasoline_A_ge_B_constraint"

# Reid vapor pressure constraints
for j, gasoline_name in enumerate(gasoline_names):
    prob += pulp.lpSum(reid[i] * x[(feedstock_names[i], gasoline_name)] for i in range(4)) <= 7 * pulp.lpSum(x[(i, gasoline_name)] for i in feedstock_names), f"Reid_vapor_pressure_{gasoline_name}_constraint"

# Octane constraints
prob += pulp.lpSum(low_tel[i] * x[(feedstock_names[i], "A")] for i in range(4)) >= 90 * pulp.lpSum(x[(i, "A")] for i in feedstock_names), "Octane_A_constraint"
prob += pulp.lpSum(high_tel[i] * x[(feedstock_names[i], "B")] for i in range(4)) >= 97 * pulp.lpSum(x[(i, "B")] for i in feedstock_names), "Octane_B_constraint"
prob += pulp.lpSum(high_tel[i] * x[(feedstock_names[i], "C")] for i in range(4)) >= 100 * pulp.lpSum(x[(i, "C")] for i in feedstock_names), "Octane_C_constraint"


### Defining Objective Function

In [7]:
revenue_gasoline = pulp.lpSum(price_gasoline[j] * pulp.lpSum(x[(i, gasoline_names[j])] for i in feedstock_names) for j in range(3))
revenue_leftover_feedstock = pulp.lpSum(price_feedstock[i] * (feedstock_avail[i] - pulp.lpSum(x[(feedstock_names[i], j)] for j in gasoline_names)) for i in range(4))
prob += revenue_gasoline + revenue_leftover_feedstock

### Problem Solving

In [8]:
prob.solve()

1

### Calculating Left Over Feedstocks

In [9]:
# Import the necessary libraries
from rich.console import Console
from rich.table import Table
from rich import print
# Initialize the console object
console = Console()
# Create a table with rich
table = Table(title="Leftover Feedstocks and Revenue Generated", show_header=True, header_style="bold")


# Calculating leftover feedstocks and revenue generated
leftover_feedstock = []
revenue_generated = []
for i, feedstock_name in enumerate(feedstock_names):
    used_feedstock = sum(x[(feedstock_name, j)].value() for j in gasoline_names)
    leftover_feedstock.append(feedstock_avail[i] - used_feedstock)
    revenue_generated.append(leftover_feedstock[i] * price_feedstock[i])

# Create a DataFrame for leftover feedstocks and revenue generated
leftover_feedstock_dict = {
    'Feedstock': feedstock_names,
    'Leftover (Gallons)': leftover_feedstock,
    'Revenue Generated ($)': revenue_generated
}
df_leftover_feedstock = pd.DataFrame(leftover_feedstock_dict)

# Round values in the DataFrame to 2 decimal points
df_leftover_feedstock = df_leftover_feedstock.round(2)


# Add columns to the table
for column in df_leftover_feedstock.columns:
    table.add_column(column)

# Add rows to the table
for _, row in df_leftover_feedstock.iterrows():
    table.add_row(*row.astype(str).tolist())


# Display the table
console.print(table)


         Leftover Feedstocks and Revenue Generated         
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Feedstock  ┃ Leftover (Gallons) ┃ Revenue Generated ($) ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Alkylate   │ 0.0                │ 0.0                   │
│ CCG        │ 42112.98           │ 105282.44             │
│ SRG        │ 0.0                │ 0.0                   │
│ Isopentane │ 62914.39           │ 147848.81             │
└────────────┴────────────────────┴───────────────────────┘

### Problem Status & Results

In [10]:

console = Console()

if pulp.LpStatus[prob.status] == 'Optimal':
    print("Optimal solution found.")

    # Creating a dictionary to store the decision variables and their values
    decision_variables = {
        'Feedstock': [],
        'Gasoline': [],
        'Gallons Used': []
    }

    # Fill the dictionary with the decision variables and their values
    for i, feedstock_name in enumerate(feedstock_names):
        for j, gasoline_name in enumerate(gasoline_names):
            decision_variables['Feedstock'].append(feedstock_name)
            decision_variables['Gasoline'].append(gasoline_name)
            decision_variables['Gallons Used'].append(x[(feedstock_name, gasoline_name)].value())

    # Converting the dictionary into a DataFrame
    df_decision_variables = pd.DataFrame(decision_variables)

    # Create a table with rich
    table = Table(title="Decision Variables", show_header=True, header_style="bold")

    # Add columns to the table
    for column in df_decision_variables.columns:
        table.add_column(column)

    # Add rows to the table
    for _, row in df_decision_variables.iterrows():
        table.add_row(*row.astype(str).tolist())

    # Display the table
    console.print(table)

    print(f"\nTotal revenue : ${prob.objective.value():,.2f}")

else:
    print("No optimal solution found.")


Optimal solution found.

           Decision Variables           
┏━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Feedstock  ┃ Gasoline ┃ Gallons Used ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Alkylate   │ A        │ 37131.064    │
│ Alkylate   │ B        │ 34312.268    │
│ Alkylate   │ C        │ 68556.668    │
│ CCG        │ A        │ 87887.023    │
│ CCG        │ B        │ 0.0          │
│ CCG        │ C        │ 0.0          │
│ SRG        │ A        │ 4899.36      │
│ SRG        │ B        │ 73457.249    │
│ SRG        │ C        │ 61643.391    │
│ Isopentane │ A        │ 82.552639    │
│ Isopentane │ B        │ 22230.483    │
│ Isopentane │ C        │ 24772.578    │
└────────────┴──────────┴──────────────┘

Total revenue : $1,718,021.80

In [11]:
a = [{'name': name, 'shadow price': c.pi, 'slack': c.slack} for name, c in prob.constraints.items()]

# Create DataFrame
df_constraints = pd.DataFrame(a)

# Create a table with rich
table = Table(title="sensitivity analysis of Constraints", show_header=True, header_style="bold")

# Add columns to the table
for column in df_constraints.columns:
    table.add_column(column)

# Add rows to the table
for _, row in df_constraints.iterrows():
    table.add_row(*row.astype(str).tolist())

# Display the table
console.print(table)

                   sensitivity analysis of Constraints                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ name                             ┃ shadow price ┃ slack               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Feedstock_Alkylate_constraint    │ 1.2665105    │ -0.0                │
│ Feedstock_CCG_constraint         │ -0.0         │ 42112.977           │
│ Feedstock_SRG_constraint         │ 0.44846025   │ -0.0                │
│ Feedstock_Isopentane_constraint  │ -0.0         │ 62914.386           │
│ Gasoline_A_constraint            │ -0.0         │ -10000.0            │
│ Gasoline_B_constraint            │ -0.38903163  │ -0.0                │
│ Gasoline_C_constraint            │ -0.0         │ -34972.640000000014 │
│ Gasoline_A_ge_B_constraint       │ -0.44037659  │ -0.0                │
│ Reid_vapor_pressure_A_constraint │ 0.27230544   │ -0.0                │
│ Reid_vapor_pressure_B_constraint │ 0.24001948   │ -0.0                │
│ Reid_vapor_pressure_C_constraint │ 0.24001948   │ -0.0                │
│ Octane_A_constraint              │ -0.22269038  │ -0.0                │
│ Octane_B_constraint              │ -0.18378165  │ 7.9092892e-12       │
│ Octane_C_constraint              │ -0.18378165  │ 7.1636916e-12       │
└──────────────────────────────────┴──────────────┴─────────────────────┘